In [8]:
import pandas as pd
import seaborn as sns
wd = "/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/Peds Endo/Petter Bjornstad/Pima/Master data/Raw data/"
# wd = "path/to/Laura's/folder"

# Vital Status
## Raw Data Info

In [37]:
# Read csv and parse dates
vital_status = pd.read_csv(wd+"NelsonVitalStatusDEC_DATA_2021-02-04_1616.csv",parse_dates=True)
# Combine rows for relevant variables
var_list = ['esrd_start_date_est','physiciancr']
for v in var_list:
    vital_status.groupby('record_id')[v].max()

In [35]:
vital_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   record_id                              583 non-null    int64  
 1   redcap_event_name                      583 non-null    object 
 2   redcap_repeat_instrument               1 non-null      object 
 3   redcap_repeat_instance                 4 non-null      float64
 4   esrd_start_date                        93 non-null     object 
 5   esrd_start_date_est                    93 non-null     float64
 6   esrd_type                              93 non-null     float64
 7   esrd_is_transplanted                   93 non-null     float64
 8   esrd_diagnosis                         93 non-null     float64
 9   esrd_complete_date                     93 non-null     object 
 10  esrd_initials                          93 non-null     object 
 11  end_st

In [27]:
max(10,nan)

NameError: name 'nan' is not defined